### **Тема “Вычисления с помощью Numpy”**



Задание 1

In [ ]:
import numpy as np
a = np.array([[1,6],
              [2,8],
              [3,11],
              [3,10],
              [1,7]],dtype=int)

mean_a = a.mean(axis=0)
print(mean_a)

[2.  8.4]


Задание 2


In [ ]:
a_centered = a - mean_a
print(a_centered)

[[-1.  -2.4]
 [ 0.  -0.4]
 [ 1.   2.6]
 [ 1.   1.6]
 [-1.  -1.4]]


Задание 3

In [ ]:
a_centered_sp = sum(a_centered[:,[0]] * a_centered[:,[1]])
N = a.shape[0]
a_centered_sp = a_centered_sp / (N-1)
print(a_centered_sp)

[2.]


Задание 4*

In [ ]:
a_cov = np.transpose(a)
a_centered_sp_2 = np.cov(a_cov, y=None, rowvar = True, bias = False)[0,1]
check = a_centered_sp_2 == a_centered_sp
print(check)

True


Задание 5**

In [ ]:
new_list = []
a_iter = a.copy()[:,[1]]
for i in range(N):
  max_val = a_iter.max()
  max_index = np.where(a == max_val)[0]
  new_list.append(max_index[0])
  max_index = np.where(a_iter == max_val)[0]
  a_iter = np.delete(a_iter, max_index)

a_sort = a[new_list,:]
print(a_sort)

[[ 3 11]
 [ 3 10]
 [ 2  8]
 [ 1  7]
 [ 1  6]]


### **Тема “Работа с данными в Pandas”**


Задание 1

In [ ]:
import pandas as pd

authors = pd.DataFrame({
    'author_id': [1, 2, 3],
    'author_name': ['Тургенев', 'Чехов', 'Островский']})

book = pd.DataFrame({'author_id': [1, 1, 1, 2, 2, 3, 3],
'book_title': ['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
'price': [450, 300, 350, 500, 450, 370, 290]
})



Задание 2

In [ ]:
authors_price = pd.merge(authors, book, on='author_id', how='right')
print(authors_price)

   author_id author_name            book_title  price
0          1    Тургенев           Отцы и дети    450
1          1    Тургенев                 Рудин    300
2          1    Тургенев     Дворянское гнездо    350
3          2       Чехов      Толстый и тонкий    500
4          2       Чехов       Дама с собачкой    450
5          3  Островский                 Гроза    370
6          3  Островский  Таланты и поклонники    290


Задание 3

In [ ]:
top5 = authors_price.nlargest(5, "price")
print(top5)

   author_id author_name         book_title  price
3          2       Чехов   Толстый и тонкий    500
0          1    Тургенев        Отцы и дети    450
4          2       Чехов    Дама с собачкой    450
5          3  Островский              Гроза    370
2          1    Тургенев  Дворянское гнездо    350


Задание 4

In [ ]:
authors_stat = authors_price.groupby("author_name").agg({'price':['min','max','mean']})
authors_stat.rename(columns = {'min' : 'min_price', 'max' : 'max_price','mean' : 'mean_price'}, inplace = True)

print(authors_stat)

                price                      
            min_price max_price  mean_price
author_name                                
Островский        290       370  330.000000
Тургенев          300       450  366.666667
Чехов             450       500  475.000000


Задание 5*

In [ ]:
authors_price ["cover"] = ['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая']

book_info = authors_price.pivot_table('price', index=['author_name'], columns= ['cover'], aggfunc = 'mean')
book_info["мягкая"] = book_info["мягкая"].fillna("0")
book_info["твердая"] = book_info["твердая"].fillna("0")
book_info.to_pickle("book_info.pkl")
print(book_info)
book_info2 = pd.read_pickle("book_info.pkl")
check = book_info == book_info2
print(check)


cover       мягкая твердая
author_name               
Островский       0     330
Тургенев       325     450
Чехов          475       0
cover        мягкая  твердая
author_name                 
Островский     True     True
Тургенев       True     True
Чехов          True     True


Задание 5**

In [242]:
import numpy as np
import pandas as pd
from datetime import date

period = {'begin': '2021-04-29', 'end': '2021-05-06'}
filters = {'sale_id': [1, 2, 3, 4], 'cost': [600, 820, 100, 700, 0]}

data = pd.DataFrame(
            [[820, '2021-04-29', 1, 213],[100, '2021-04-29', 1, 213],
             [600, '2021-04-30', 2, 113],[700, '2021-05-01', 3, 893],[0, '2021-05-03',4,0]],
            columns=['cost', 'date', 'sale_id', 'shop_id']
        )

def calculate_sales_metrics(df, cost_name, date_name, sale_id_name, shop_id_name, period, filters=None):
  df[date_name]= pd.to_datetime(df[date_name])

  period_filter = pd.Series(period, index=list(period.keys()))
  end = pd.Timestamp(period_filter['end'])
  begin = pd.Timestamp(period_filter['begin'])
  
  condition = ((df[sale_id_name].isin(filters['sale_id'])) & 
               (df[cost_name].isin(filters['cost'])) & 
               (df[date_name] >= begin) & 
               (df[date_name] < end))
  new_data = df[condition]
 
  new_data = new_data.groupby(date_name).agg({cost_name:['sum','mean'], sale_id_name:['count'], shop_id_name:['mean']})
  new_data.sort_values(by=date_name, inplace=True)
  new_data['cost'] = new_data['cost'].fillna("0")
  new_data['sale_id'] = new_data['sale_id'].fillna("0")
  new_data['shop_id'] = new_data['shop_id'].fillna("0")
  new_data.rename(columns = {'sum' : 'revenue', 'count' : 'number_purchases','mean' : 'average_check','mean' : 'average_number_items'}, inplace = True)
  
  return new_data

calculate_sales_metrics(data,'cost','date',
                        'sale_id','shop_id',period,filters)

cost                               sale_id              shop_id
           revenue average_number_items number_purchases average_number_items
date                                                                         
2021-04-29     920                  460                2                  213
2021-04-30     600                  600                1                  113
2021-05-01     700                  700                1                  893
2021-05-03       0                    0                1                    0